Trabalhando com dados categoricos para fazer a previsão de riscos com base nessa mesma coluna

In [122]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.neighbors import KNeighborsClassifier


In [123]:
df = pd.read_csv('https://raw.githubusercontent.com/ronaldolagepessoa/machine-learning/master/dados/exemplo3.csv')

In [124]:
#Trabalhando com dados categóricos ['sexo']
df.head()

,idade,conta_corrente,sexo,risco
0,21.257389,783.127911,masculino,ruim
1,21.766573,979.747521,feminino,bom
2,25.321033,1065.328054,feminino,bom
3,23.919268,1195.758078,feminino,bom
4,21.805298,1083.764450,feminino,bom


In [125]:
X = df.drop('risco', axis=1)
y = df.risco

y.head()

0    ruim
1     bom
2     bom
3     bom
4     bom
Name: risco, dtype: object

In [126]:
#Função para selecionar um dato categórico (object)

X.select_dtypes(include='object')

,sexo
0,masculino
1,feminino
2,feminino
3,feminino
4,feminino
...,...
2495,feminino
2496,masculino
2497,masculino
2498,masculino


In [127]:
#Guardado em uma variavel esses dados categoricos da planilha
X_cat = X.select_dtypes(include='object')

In [128]:
#Vamos fazer uma processo de binarização utilizando OneHotEncoder
from sklearn.preprocessing import OneHotEncoder 

onehot = OneHotEncoder(sparse=False, drop='first') #chamando a function com os parametros 'sparse' para o resultado não ser uma matriz dessa forma e 'drop' para eliminar uma das colunas geradas
X_bin = onehot.fit_transform(X_cat)


In [129]:
#Fazendo a normalização das colunas numéricas
from sklearn.preprocessing import MinMaxScaler

X_num = X.select_dtypes(exclude='object')
minmax = MinMaxScaler()

X_num = minmax.fit_transform(X_num)

In [130]:
#Function do numpy que ajuda a unir os dois valores já processados em um unico DF
X_all = np.append(X_num, X_bin, axis=1)

In [131]:
#Importando os pacotes para auxiliar nas precições
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split  #divisão de conjunto de treinamento e conjunto de teste
from sklearn.metrics import accuracy_score #medida de precisão do nosso classificador

#Divisão dos conjuntos de treinamento e testes
X_train, X_test, y_train, y_test = train_test_split(X_all, y , test_size=1/3, random_state=42)

#Treinamento do classificador SVM
svc2 = SVC()
svc2.fit(X_train, y_train)

#Cálculo da precisão
accuracy_score(y_test, svc2.predict(X_test))

0.7146282973621103

In [132]:
#Verificação com um novo conjunto de dados

df_new = pd.DataFrame({
    'idade': [20,25,50,35,75],
    'conta_corrente': [800,400,2200,3200,1000],
    'sexo': ['masculino', 'feminino', 'masculino', 'feminino', 'feminino']
})

df_new

,idade,conta_corrente,sexo
0,20,800,masculino
1,25,400,feminino
2,50,2200,masculino
3,35,3200,feminino
4,75,1000,feminino


In [133]:
#Fazendo a previsão de risco do novo DF, para isso precisamos utilizar o mesmo formato de treinamento do classificador
X_new_bin = onehot.transform(df_new.select_dtypes(include=['object']))
X_new_num = minmax.transform(df_new.select_dtypes(exclude=['object']))
X_new = np.append(X_new_num, X_new_bin, axis=1)

svc2.predict(X_new) 

array(['ruim', 'bom', 'ruim', 'bom', 'bom'], dtype=object)

In [134]:
#Mostrando o resultado em forma de DataFrame, precisamos apenas criar uma nova coluna contendo os resultados da previsão.
df_previsao = df_new.copy()
df_previsao['previsao'] = svc2.predict(X_new)
df_previsao

,idade,conta_corrente,sexo,previsao
0,20,800,masculino,ruim
1,25,400,feminino,bom
2,50,2200,masculino,ruim
3,35,3200,feminino,bom
4,75,1000,feminino,bom
